In [1]:
# Author : Arthur Prigent
# Email : aprigent@geomar.de

In [5]:
from load_librairies import *
import xscale.signal.fitting as xsf
import numpy as np
import scipy.interpolate as sc_interp
import warnings
import xarray as xr
warnings.warn('ignore')
dir_T = '/projects/NS9039K/shared/CMIP_data/CMIP5_1x1_mon/Omon/historical/thetao/mon/'
#dir_out = '/projects/NS9039K/shared/CMIP_data/CMIP5_1x1_mon/Omon/historical/z20/eqAtl/'
now = datetime.now()
date_time = now.strftime("%d/%m/%Y")
lon_min = -20
lon_max = 0
lat_min = -3
lat_max = 3

period_str_1 = 1950
period_end_1 = 1999

period_str_2 = 2050
period_end_2 = 2099

def is_jja(month):
    return (month >= 6) & (month <= 8)

def isotherm(temp,Z,isovalue):
    '''
    It is constant temperature depth. i.e. 20 deg. C isotherm defines depth at which temperature 
    is 20C, often denoted as Z20 or D20 
    ==========================================================%
    
    USAGE: isotherm=ra_isotherm(temp,Z,isovalue)
    
    DESCRIPTION:  This function determines Isotherms from profile data sets. If you have 3D 
    data sets i.e. level, lat and lon and want to compute the Isotherm, then this function will 
    be very handy. Because this function is specifically designed for those cases. However, 
    it can evaluate isotherms from profile data too.
    
    INPUTS: 
    temp = Temperature profiles over the study region [deg. C], either 3D or vector
    Z = Levels [m], Must be vector
    isovalue = temperature at which you want to compute isotherms [deg. C], Must be scalar
    
    OUTPUT: 
    isotherm = Isotherms depth, spatial output [m]
    '''

    (lv, lt, ln)=temp.shape
    T = np.array(temp).reshape(lv,lt*ln)
    #print(T.shape)
    oce= np.where(temp[0,:,:]!=np.nan)
    land = np.where(temp[0,:,:]==np.nan)
    T[:,land] = np.nan
    #print(oce)
    (_,nloop)=T.shape
    #print(nloop)
    therm = np.ones((nloop))*np.nan
    for i in range(nloop):
        t = T[:,i]
        id1 = np.array(np.where(t < isovalue)).ravel()
        
        if (len(id1) > 0) and (id1[0]>1) :
            
            p2 = id1[0]
            p1 = p2-1
            f = sc_interp.interp1d(t[[p1,p2]],Z[[p1,p2]])
            therm[i] = f(isovalue)
        else:
            therm[i] = np.nan
            
    isotherm = np.ones((lt*ln))*np.nan
    #isotherm[oce] = therm
    isotherm = isotherm.reshape(lt,ln)
    isotherm[oce] = therm
    return isotherm


def data_sub(data,lon_min,lon_max,lat_min,lat_max):
    
    '''Define a box between lon_min lon_max lat_min and lat_max and 
    extract the data in the box and drop everything else.
    
    
    Parameters
    ----------
    
    data : xarray_like
    Data to be subdomained. 
    
    lon_min : integer
    Longitude minimum of the subdomain
    
    lon_max : integer
    Longitude maximum of the subdomain
    
    lat_min : integer
    Latitude minimum of the subdomain
    
    lat_max : integer
    Latitude maximum of the subdomain
    
    Returns
    ---------
    
    data_sub : xarray_like
    Subdomain. 
    '''
    
    try:
        data_sub = data.where((  data.lon>=lon_min) & (data.lon<=lon_max) & (data.lat<=lat_max) & (data.lat>=lat_min),
                                                                          drop=True)
    except AttributeError:
        try:
            data_sub = data.where((  data.nav_lon>=lon_min) & (data.nav_lon<=lon_max) & (data.nav_lat<=lat_max) & (data.nav_lat>=lat_min),drop=True)
        except AttributeError:
            try:
                data_sub = data.where((  data.longitude>=lon_min) & (data.longitude<=lon_max) & (data.latitude<=lat_max) & (data.latitude>=lat_min),drop=True)
            except AttributeError:
                try:
                    data_sub = data.where((  data.x>=lon_min) & (data.x<=lon_max) & (data.y<=lat_max) &
                                      (data.y>=lat_min),drop=True)
                except AttributeError:
                    try:
                        data_sub = data.where((  data.LON>=lon_min) & (data.LON<=lon_max) & (data.LAT<=lat_max) &
                                      (data.LAT>=lat_min),drop=True)
                    except AttributeError:
                        data_sub = data.where((  data.LONGITUDE>=lon_min) & (data.LONGITUDE<=lon_max) &
                                                  (data.LATITUDE<=lat_max) &(data.LATITUDE>=lat_min),drop=True)
            
    

 
    
    return data_sub
dir_data ='/Volumes/Arthur_disk/Thesis_GEOMAR/Data/CMIP5/hist/thetao/eqatl/'
dir_data2 ='/Volumes/Arthur_disk/Thesis_GEOMAR/Data/CMIP5/rcp85/thetao/eqatl/'
import cftime
import datetime

path_data = '/Users/aprigent/Documents/Thesis_GEOMAR/Projects/weakened_sst_variability_CMIP5/scripts/new_version/to_publish/data/'

/Users/aprigent/miniconda3/envs/py3_std/lib/python3.7/site-packages/ipykernel_launcher.py:7: UserWarning: ignore
  import sys


In [3]:
model = ['ACCESS1-0', 'ACCESS1-3','CMCC-CESM',
         'CMCC-CMS','CMCC-CM','CNRM-CM5','CSIRO-Mk3-6-0',
        'GFDL-CM3', 'GFDL-ESM2M','GISS-E2-H-CC','GISS-E2-H',
        'GISS-E2-R-CC','GISS-E2-R','HadGEM2-CC','IPSL-CM5A-LR',
        'IPSL-CM5A-MR','IPSL-CM5B-LR','MIROC-ESM-CHEM','MIROC-ESM',
        'MIROC5','MPI-ESM-LR','MPI-ESM-MR','MRI-CGCM3','NorESM1-M']
len(model)

24

In [6]:
temp_cmip5_hist = np.ones((len(model),56,51))*np.nan
temp_cmip5_hist_jja = np.ones((len(model),56,51))*np.nan

for j in range(len(model)):
    print(model[j])
    data_tmp = xr.open_dataset(dir_data+'thetao_Omon_'+model[j]+'_historical_r1i1p1_187001-200012_1deg_vertint_eqatl.nc')

    temp = data_tmp.thetao[:,:,:,:]-273.15

    try:
        temp = temp.sel(time=slice(datetime.datetime(period_str_1, 1, 1),
                                   datetime.datetime(period_end_1, 12, 31)))
        
    except TypeError:
        try:
            
            temp['time'] = temp['time'].to_datetimeindex()
            temp = temp.sel(time=slice(datetime.datetime(period_str_1, 1, 1),
                                       datetime.datetime(period_end_1, 12, 31)))
        except AttributeError:
            try:
                temp = temp.sel(time=slice(cftime.DatetimeNoLeap(period_str_1, 1, 1, 1, 0, 0, 0, has_year_zero=True),
                                       cftime.DatetimeNoLeap(period_end_1, 1, 1, 1, 0, 0, 0, has_year_zero=True)))
            except TypeError:
                temp = temp.sel(time=slice(cftime.Datetime360Day(period_str_1, 1, 1, 1, 0, 0, 0, has_year_zero=True),
                                       cftime.Datetime360Day(period_end_1, 1, 1, 1, 0, 0, 0, has_year_zero=True)))

    temp = temp.where(temp>0)
    temp_mean_jja = temp.sel(time=is_jja(temp['time.month'])).mean(dim='time').mean(dim='lat')
    temp_mean = temp.mean(dim='time').mean(dim='lat')
    
    temp_cmip5_hist[j,:,:] = np.array(temp_mean)
    temp_cmip5_hist_jja[j,:,:] = np.array(temp_mean_jja)

ACCESS1-0
ACCESS1-3
CMCC-CESM
CMCC-CMS
CMCC-CM
CNRM-CM5
CSIRO-Mk3-6-0
GFDL-CM3
GFDL-ESM2M
GISS-E2-H-CC
GISS-E2-H
GISS-E2-R-CC
GISS-E2-R
HadGEM2-CC
IPSL-CM5A-LR
IPSL-CM5A-MR
IPSL-CM5B-LR
MIROC-ESM-CHEM
MIROC-ESM
MIROC5
MPI-ESM-LR
MPI-ESM-MR
MRI-CGCM3
NorESM1-M


In [7]:
temp_eq  = xr.Dataset({'temp_eq': (['model','depth','lon'],temp_cmip5_hist),
                           'temp_eq_jja': (['model','depth','lon'],temp_cmip5_hist_jja)}
                       ,coords={'model':model,
                          'depth':np.array(temp.lev),
                               'lon':np.array(temp.lon)},attrs={'standard_name': 'Temp',
                                    'long_name': 'Equatorial Temperature',
                                    'units': 'K',
                                    'model': 'CMIP5',
                                    'Scenario': 'HIST',
                                    'Creation_date':date_time,   
                                    'author': 'Arthur Prigent'})

In [8]:
temp_eq.to_netcdf(path_data+'temp_eq_cmip5_40W_10E_hist.nc')

# 2050 2099

In [9]:
temp_cmip5_rcp = np.ones((len(model),56,51))*np.nan
temp_cmip5_rcp_jja = np.ones((len(model),56,51))*np.nan

for j in range(len(model)):
    print(model[j])
    
    data_tmp = xr.open_dataset(dir_data2+'thetao_Omon_'+model[j]+'_rcp85_r1i1p1_201501-209912_1deg_vertint_eqatl.nc')

    temp = data_tmp.thetao[:,:,:,:]-273.15

    try:
        temp = temp.sel(time=slice(datetime.datetime(period_str_2, 1, 1),
                                   datetime.datetime(period_end_2, 12, 31)))
        
    except TypeError:
        try:
            
            temp['time'] = temp['time'].to_datetimeindex()
            temp = temp.sel(time=slice(datetime.datetime(period_str_2, 1, 1),
                                       datetime.datetime(period_end_2, 12, 31)))
        except AttributeError:
            try:
                temp = temp.sel(time=slice(cftime.DatetimeNoLeap(period_str_2, 1, 1, 1, 0, 0, 0, has_year_zero=True),
                                       cftime.DatetimeNoLeap(period_end_2, 1, 1, 1, 0, 0, 0, has_year_zero=True)))
            except TypeError:
                temp = temp.sel(time=slice(cftime.Datetime360Day(period_str_2, 1, 1, 1, 0, 0, 0, has_year_zero=True),
                                       cftime.Datetime360Day(period_end_2, 1, 1, 1, 0, 0, 0, has_year_zero=True)))

    temp = temp.where(temp>0)
    temp_mean_jja = temp.sel(time=is_jja(temp['time.month'])).mean(dim='time').mean(dim='lat')
    temp_mean = temp.mean(dim='time').mean(dim='lat')
    
    temp_cmip5_rcp[j,:,:] = np.array(temp_mean)
    temp_cmip5_rcp_jja[j,:,:] = np.array(temp_mean_jja)

ACCESS1-0
ACCESS1-3
CMCC-CESM
CMCC-CMS
CMCC-CM
CNRM-CM5
CSIRO-Mk3-6-0
GFDL-CM3
GFDL-ESM2M
GISS-E2-H-CC
GISS-E2-H
GISS-E2-R-CC
GISS-E2-R
HadGEM2-CC
IPSL-CM5A-LR
IPSL-CM5A-MR
IPSL-CM5B-LR
MIROC-ESM-CHEM
MIROC-ESM
MIROC5
MPI-ESM-LR
MPI-ESM-MR
MRI-CGCM3
NorESM1-M


In [10]:
temp_eq_rcp85  = xr.Dataset({'temp_eq': (['model','depth','lon'],temp_cmip5_rcp),
                           'temp_eq_jja': (['model','depth','lon'],temp_cmip5_rcp_jja)}
                       ,coords={'model':model,
                          'depth':np.array(temp.lev),
                               'lon':np.array(temp.lon)},attrs={'standard_name': 'Temp',
                                    'long_name': 'Equatorial Temperature',
                                    'units': 'K',
                                    'model': 'CMIP5',
                                    'Scenario': 'RCP85',
                                    'Creation_date':date_time,   
                                    'author': 'Arthur Prigent'})

In [11]:
temp_eq_rcp85.to_netcdf(path_data+'temp_eq_cmip5_40W_10E_rcp85.nc')